In [2]:
import dill
import pandas as pd
import altair as alt
import numpy as np

In [5]:
d_all_EOD = dill.load(open('/home/flor/cs236/data/d_all_EOD.pkd','rb'))
for ticker in d_all_EOD.keys():
    d_all_EOD[ticker] = d_all_EOD[ticker].reset_index(drop=True)

In [30]:
def find_low(ticker, date):
    df_truth = d_all_EOD[ticker]
    df1 = df_truth[df_truth.date==date]
    index = df1.index[0]
    return df_truth.iloc[index-1]['low']

def find_high(ticker,date):
    df_truth = d_all_EOD[ticker]
    df1 = df_truth[df_truth.date==date]
    index = df1.index[0]
    return df_truth.iloc[index+1]['high']

def calc_pe(prediction,true_value):
    return 100*(prediction - true_value)/true_value

def calc_ape(prediction,true_value):
    return 100*np.abs(prediction - true_value)/true_value
    
def calc_mpe(d):
    l_temp =[]
    for ticker, dates in d.items():
        for date, df in dates.items():
            true_low = find_low(ticker,date)
            true_high = find_high(ticker,date)
            l_temp.append({'date': date, 'ticker': ticker,
                        'pe_low': calc_pe(d[ticker][date]['low'], true_low), 
                        'pe_high': calc_pe(d[ticker][date]['high'], true_high),
                        'ape_low': calc_ape(d[ticker][date]['low'], true_low),
                        'ape_high': calc_ape(d[ticker][date]['high'], true_high)})
    df_results = pd.DataFrame(l_temp)
    
    print('MPE Low:',df_results.pe_low.mean())
    print('MPE High:',df_results.pe_high.mean())
    print('MPE Overall:',df_results[['pe_low','pe_high']].mean().mean())
    print('MAPE Low:',df_results.ape_low.mean())
    print('MAPE High:',df_results.ape_high.mean())
    print('MAPE Overall:',df_results[['ape_low','ape_high']].mean().mean())
    
    return df_results.sort_values('date').reset_index(drop=True)

def show_results(predictions_path):
    df = calc_mpe(dill.load(open(predictions_path,'rb')))
    return alt.Chart(df,title='PE for Low and High').mark_line().encode(x='date',y='pe_low')&\
alt.Chart(df).mark_line().encode(x='date',y='pe_high')

### CVAE predictions - baseline

In [32]:
show_results('/home/flor/cs236/data/predictions_cvae83.pkd')

MPE Low: 1.640174301338967
MPE High: 2.4722017017146234
MPE Overall: 2.0561880015267953
MAPE Low: 11.94763298855777
MAPE High: 11.546397276446879
MAPE Overall: 11.747015132502323


alt.VConcatChart(...)

### CVAE Predictions - With Sentiment

In [33]:
show_results('/home/flor/cs236/data/predictions_cvae83_sent.pkd')

MPE Low: 0.43418693994337965
MPE High: 1.4903526315177495
MPE Overall: 0.9622697857305647
MAPE Low: 11.109080723798684
MAPE High: 10.924856218318737
MAPE Overall: 11.01696847105871


alt.VConcatChart(...)

### NN Predictions with Sentiment

In [34]:
show_results('/home/flor/cs236/data/predictions_baseline83_sent.pkd')

MPE Low: -0.03842297908281038
MPE High: 0.20082292371585528
MPE Overall: 0.08119997231652246
MAPE Low: 2.2057702933255943
MAPE High: 1.5927511357294062
MAPE Overall: 1.8992607145275002


alt.VConcatChart(...)

### CVAE Predictions with input 10

In [35]:
show_results('/home/flor/cs236/data/predictions_cvae83_input10.pkd')

MPE Low: 0.3069754766659162
MPE High: 1.4303366552735768
MPE Overall: 0.8686560659697464
MAPE Low: 11.166685742180084
MAPE High: 10.955047526077067
MAPE Overall: 11.060866634128576


alt.VConcatChart(...)

### CVAE Sentiment predictions with input 10 and z 2

In [36]:
show_results('/home/flor/cs236/data/predictions_sent_z2.pkd')

MPE Low: 0.35036088973434787
MPE High: 1.3334183175911967
MPE Overall: 0.8418896036627723
MAPE Low: 11.094798771015185
MAPE High: 10.956101102006192
MAPE Overall: 11.02544993651069


alt.VConcatChart(...)

In [37]:
show_results('/home/flor/cs236/data/predictions_baseline_sent_z2.pkd')

MPE Low: -0.1595979094609247
MPE High: 0.06595943911385689
MPE Overall: -0.0468192351735339
MAPE Low: 1.8332328632268495
MAPE High: 1.6969414177284359
MAPE Overall: 1.7650871404776427


alt.VConcatChart(...)

In [38]:
show_results('/home/flor/cs236/data/arima_predictions.pkd')

MPE Low: 0.2137427129354803
MPE High: -0.14773150859885706
MPE Overall: 0.033005602168311624
MAPE Low: 1.1605152450069154
MAPE High: 1.1084932843579358
MAPE Overall: 1.1345042646824255


alt.VConcatChart(...)

In [39]:
show_results('/home/flor/cs236/data/predictions_cvae83_z10.pkd')

MPE Low: 0.36793170979237927
MPE High: 1.2210996244904069
MPE Overall: 0.7945156671413931
MAPE Low: 11.012726632603453
MAPE High: 10.839363731365935
MAPE Overall: 10.926045181984694


alt.VConcatChart(...)

In [40]:
show_results('/home/flor/cs236/data/predictions_baseline83_z10.pkd')

MPE Low: -0.03421544519704279
MPE High: 0.16005980566115874
MPE Overall: 0.06292218023205798
MAPE Low: 2.152021283384022
MAPE High: 1.399529700665535
MAPE Overall: 1.7757754920247786


alt.VConcatChart(...)

In [41]:
show_results('/home/flor/cs236/data/dvae_predictions.pkd')

MPE Low: -0.20167922579068895
MPE High: 0.5895755676384965
MPE Overall: 0.19394817092390376
MAPE Low: 0.7822775094655949
MAPE High: 1.7442639096583226
MAPE Overall: 1.2632707095619589


alt.VConcatChart(...)